# Explore here

In [1]:
import os
import pandas as pd
import seaborn as sns
from dotenv import load_dotenv

# load the .env file variables
load_dotenv()



True

In [2]:
client_id = os.environ.get("CLIENT_ID")
client_secret = os.environ.get("CLIENT_SECRET")

In [3]:
import base64
import requests

# Encode the client credentials
client_credentials = f"{client_id}:{client_secret}"
client_credentials_base64 = base64.b64encode(client_credentials.encode())

# Prepare the token request
token_url = "https://accounts.spotify.com/api/token"
headers = {
    "Authorization": f"Basic {client_credentials_base64.decode()}"
}
data = {
    "grant_type": "client_credentials"
}

# Request access token
response = requests.post(token_url, headers=headers, data=data)

# Check if the request was successful
if response.status_code == 200:
    access_token = response.json()['access_token']
    print("Access token obtained!")
else:
    print(f"Failed to obtain access token. Status code: {response.status_code}")
    print(response.text)

Access token obtained!


In [4]:
# Set the access token in the headers
headers = {
    "Authorization": f"Bearer {access_token}"
}

# Define the endpoint and parameters
search_url = "https://api.spotify.com/v1/search"
params = {
    "q": "Drake",  # Artist name to search for
    "type": "artist",
    "limit": 1
}

# Make the API request to search for the artist
response = requests.get(search_url, headers=headers, params=params)

# Check if the request was successful
if response.status_code == 200:
    response_data = response.json()
    # print(f"Response Data: {response_data}")
    artist_id = response_data['artists']['items'][0]['id']
    artist_name = response_data['artists']['items'][0]['name']
    print(f"Found artist: {artist_name} (ID: {artist_id})")
else:
    print(f"Failed to search for artist. Status code: {response.status_code}")
    print(response.text)

Found artist: Drake (ID: 3TVXtAsR1Inumwj472S9r4)


In [5]:
# Define the endpoint for the artist's top tracks
top_tracks_url = f"https://api.spotify.com/v1/artists/{artist_id}/top-tracks"
params = {
    "market": "US"  # Specify the market (country)
}

# Make the API request to get top tracks
response = requests.get(top_tracks_url, headers=headers, params=params)

# Check if the request was successful
if response.status_code == 200:
    top_tracks_data = response.json()
    print(f"Top tracks for {artist_name}:")
    for idx, track in enumerate(top_tracks_data['tracks'], start=1):
        # print(f"Track data: {track}")
        track_name = track['name']
        album_name = track['album']['name']
        print(f"{idx}. {track_name} - Album: {album_name}")
else:
    print(f"Failed to get top tracks. Status code: {response.status_code}")
    print(response.text)

Top tracks for Drake:
1. One Dance - Album: Views
2. WAIT FOR U (feat. Drake & Tems) - Album: I NEVER LIKED YOU
3. God's Plan - Album: Scorpion
4. Passionfruit - Album: More Life
5. Jimmy Cooks (feat. 21 Savage) - Album: Honestly, Nevermind
6. Circadian Rhythm - Album: 100 GIGS
7. No Face - Album: No Face
8. Rich Baby Daddy (feat. Sexyy Red & SZA) - Album: For All The Dogs
9. Fair Trade (with Travis Scott) - Album: Certified Lover Boy
10. It's Up (feat. Young Thug & 21 Savage) - Album: 100 GIGS


In [6]:
top_tracks_data

{'tracks': [{'album': {'album_type': 'album',
    'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3TVXtAsR1Inumwj472S9r4'},
      'href': 'https://api.spotify.com/v1/artists/3TVXtAsR1Inumwj472S9r4',
      'id': '3TVXtAsR1Inumwj472S9r4',
      'name': 'Drake',
      'type': 'artist',
      'uri': 'spotify:artist:3TVXtAsR1Inumwj472S9r4'}],
    'external_urls': {'spotify': 'https://open.spotify.com/album/40GMAhriYJRO1rsY4YdrZb'},
    'href': 'https://api.spotify.com/v1/albums/40GMAhriYJRO1rsY4YdrZb',
    'id': '40GMAhriYJRO1rsY4YdrZb',
    'images': [{'url': 'https://i.scdn.co/image/ab67616d0000b2739416ed64daf84936d89e671c',
      'width': 640,
      'height': 640},
     {'url': 'https://i.scdn.co/image/ab67616d00001e029416ed64daf84936d89e671c',
      'width': 300,
      'height': 300},
     {'url': 'https://i.scdn.co/image/ab67616d000048519416ed64daf84936d89e671c',
      'width': 64,
      'height': 64}],
    'is_playable': True,
    'name': 'Views',
    'relea

In [7]:
def get_access_token(client_id, client_secret):
    try:
        # Encoding and token request as before
        client_credentials = f"{client_id}:{client_secret}"
        client_credentials_base64 = base64.b64encode(client_credentials.encode())
        token_url = "https://accounts.spotify.com/api/token"
        headers = {
            "Authorization": f"Basic {client_credentials_base64.decode()}"
        }
        data = {
            "grant_type": "client_credentials"
        }
        response = requests.post(token_url, headers=headers, data=data)
        response.raise_for_status()  # Raises an HTTPError for bad status codes
        access_token = response.json()['access_token']
        return access_token
    except requests.exceptions.HTTPError as err:
        print(f"HTTP error occurred: {err}")
    except Exception as err:
        print(f"An error occurred: {err}")
    return None

# Use the function to get the access token
access_token = get_access_token(client_id, client_secret)
if access_token:
    print("Access token obtained successfully.")
else:
    print("Failed to obtain access token.")

Access token obtained successfully.


In [8]:
track_list = []
for track in top_tracks_data['tracks']:
    track_info = {
        'Track ': track['name'],
        'Album ': track['album']['name'],
        'Popularity': track['popularity'],
        'Duration (min)': track['duration_ms'] / 60000
    } 
    track_list.append(track_info)
df = pd.DataFrame(track_list)
print(df)

                                    Track                Album   Popularity  \
0                                One Dance                Views          85   
1          WAIT FOR U (feat. Drake & Tems)    I NEVER LIKED YOU          82   
2                               God's Plan             Scorpion          81   
3                             Passionfruit            More Life          80   
4            Jimmy Cooks (feat. 21 Savage)  Honestly, Nevermind          80   
5                         Circadian Rhythm             100 GIGS          79   
6                                  No Face              No Face          79   
7  Rich Baby Daddy (feat. Sexyy Red & SZA)     For All The Dogs          79   
8           Fair Trade (with Travis Scott)  Certified Lover Boy          78   
9   It's Up (feat. Young Thug & 21 Savage)             100 GIGS          77   

   Duration (min)  
0        2.899767  
1        3.164883  
2        3.316217  
3        4.982333  
4        3.639400  
5        2

In [9]:
df

,Track,Album,Popularity,Duration (min)
0,One Dance,Views,85,2.899767
1,WAIT FOR U (feat. Drake & Tems),I NEVER LIKED YOU,82,3.164883
2,God's Plan,Scorpion,81,3.316217
3,Passionfruit,More Life,80,4.982333
4,Jimmy Cooks (feat. 21 Savage),"Honestly, Nevermind",80,3.639400
5,Circadian Rhythm,100 GIGS,79,2.105933
6,No Face,No Face,79,2.285700
7,Rich Baby Daddy (feat. Sexyy Red & SZA),For All The Dogs,79,5.319850
8,Fair Trade (with Travis Scott),Certified Lover Boy,78,4.852917
9,It's Up (feat. Young Thug & 21 Savage),100 GIGS,77,4.633333
